<a href="https://colab.research.google.com/github/sbjglitch/Sanskrit2mal/blob/main/sanskrit_to_malayalam_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

#%pip install inltk
#%pip install python-utils
%pip install utils
import inltk 
from inltk.inltk import setup
from inltk.inltk import tokenize

import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
#from utils import bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Note: you may need to restart the kernel to use updated packages.


In [118]:
%cd C:\Users\Glitch\Sanskrit2mal

C:\Users\Glitch\Sanskrit2mal


In [119]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus

mal_lines = []
sanskrit_lines = []
with open ('bhagavadgita_malayalam.txt','r', encoding="utf8") as f:
  mal_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open ('bhagvadgita_sanskrit.txt','r',encoding="utf8") as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

print(len(mal_lines))
print(len(sanskrit_lines))

C:\Users\Glitch\Sanskrit2mal\parallel-corpus
701
701


In [120]:
import random
c = list(zip(sanskrit_lines, mal_lines))
random.shuffle(c)

sanskrit_lines, mal_lines = zip(*c)

train_text_sa = sanskrit_lines[:631]
train_text_mal = mal_lines[:631]

dev_text_sa = sanskrit_lines[:70]
dev_text_mal = mal_lines[:70]

In [121]:
print(len(train_text_mal))
print(len(dev_text_mal))

print(len(train_text_sa))
print(len(dev_text_sa))

631
70
631
70


In [92]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [93]:
# Reading the data and forming pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_mal[i]] for i in range(len(train_text_sa))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [94]:
MAX_LENGTH = 200

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [95]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'mal', False)
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 701 sentence pairs
Trimmed to 701 sentence pairs
Counting words...
Counted words:
sanskrit 4342
mal 6297
['यो यो यां यां तनुं भक्तः श्रद्धयार्चितुमिच्छति। तस्य तस्याचलां श्रद्धां तामेव विदधाम्यहम्।।', 'ഏതേതു ഭക്തന്\u200d ഏതേതു ദേവതാ സ്വരൂപത്തെ ശ്രദ്ധയോടെ അര്\u200dച്ചിക്കാനാഗ്രഹിക്കുന്നുവോ അവരുടെയെല്ല\u200dാം ആ ശ്രദ്ധയെ തന്നെ ഞാന്\u200d ഇളക്കമറ്റതാക്കുന്നു.']


In [96]:
print(input_lang)

# **Seq2Seq Model**
---
A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

# **Encoder**

In [97]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Decoder**

In [98]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# **Training**


In [99]:
import random
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(list(lang.word2index.values()))    
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# **Training the model**

In [100]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [101]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [102]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    #training_pairs = training_pairs1
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

# **Evaluation**

In [103]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [104]:
def evaluate_test(encoder, decoder):
  test = []
  %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
  with open('test_sanskrit.sa', 'r', encoding="utf8") as f:
    test_text = [x.replace('\n', '') for x in f.readlines()]
  for sa_text in test_text:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    # print('<', output_sentence)
    # print('')
    test.append(output_sentence)
  with open('translated_malayalam.txt', 'w', encoding="utf8") as f:
    for line in test:
        f.write(line + '\n')

In [105]:
def translator(encoder, decoder):
    test_text = 'धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः। मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय।।'

    output_words, attentions = evaluate(encoder, decoder, test_text)
    output_sentence = ' '.join(output_words)
    print(output_sentence)

In [106]:
 %cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [112]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 75000, print_every=500)

0m 17s (- 44m 20s) (500 0%) 5.6057
0m 31s (- 38m 15s) (1000 1%) 6.3284
0m 44s (- 36m 28s) (1500 2%) 6.2658
0m 58s (- 35m 34s) (2000 2%) 6.4010
1m 12s (- 34m 55s) (2500 3%) 6.2947
1m 26s (- 34m 24s) (3000 4%) 6.3178
1m 39s (- 34m 2s) (3500 4%) 6.1887
1m 53s (- 33m 42s) (4000 5%) 5.9878
2m 8s (- 33m 29s) (4500 6%) 5.9296
2m 22s (- 33m 16s) (5000 6%) 5.6801
2m 36s (- 33m 3s) (5500 7%) 5.6055
2m 51s (- 32m 51s) (6000 8%) 5.1771
3m 6s (- 32m 43s) (6500 8%) 4.8597
3m 20s (- 32m 28s) (7000 9%) 4.5984
3m 35s (- 32m 19s) (7500 10%) 4.3647
3m 50s (- 32m 6s) (8000 10%) 3.8246
4m 5s (- 31m 57s) (8500 11%) 3.3958
4m 20s (- 31m 48s) (9000 12%) 3.2928
4m 35s (- 31m 41s) (9500 12%) 2.8090
4m 51s (- 31m 31s) (10000 13%) 2.6398
5m 6s (- 31m 21s) (10500 14%) 2.2508
5m 22s (- 31m 15s) (11000 14%) 2.0546
5m 38s (- 31m 7s) (11500 15%) 1.7268
5m 53s (- 30m 56s) (12000 16%) 1.5477
6m 9s (- 30m 45s) (12500 16%) 1.3701
6m 24s (- 30m 34s) (13000 17%) 1.0805
6m 40s (- 30m 25s) (13500 18%) 0.8965
6m 57s (- 30m 17s

In [113]:
# saving the trained models
%cd C:\Users\Glitch\Sanskrit2mal
torch.save(encoder1.state_dict(), 'trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), 'trained_decoder.pth')

C:\Users\Glitch\Sanskrit2mal


In [114]:
evaluate_test(encoder1, attn_decoder1)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal


In [115]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
output_text=[]
with open ('translated_malayalam.txt','r',encoding="utf8") as f:
  output_text.extend([f.readlines()])
print(output_text)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
[['ധൃതരാഷ്ട്രര്\u200d ചോദിച്ചു: സഞ്ജയ, ധ\u200dര്\u200dമ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്\u200d ഒരുമിച്ചു ചേര്\u200dന്നവരും യുദ്ധം ചെയ്യാന്\u200d കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്\u200dമാരും പാണ്ഡവന്\u200dമാരും എന്ത് ചെയ്തു? <EOS>\n', 'സഞ്ജയന്\u200d പറഞ്ഞു: അണിനിരന്ന പാണ്ഡവ സൈന്യത്തെ കണ്ടിട്ട് രാജാവായ ദുര്യോധനന്\u200d ദ്രോണാചാര്യരെ സമീപിച്ചു പറഞ്ഞു. <EOS>\n', 'ഹേ ആചാര്യാ, അങ്ങയുടെ ശിഷ്യനും ബുദ്ധിമാനുമായ ദ്രു\u200cപദപുത്രനാല്\u200d അണിനിരത്തപ്പെട്ട പാണ്ഡവന്\u200dമാരുടെ ഈ വലിയ സൈന്യത്തെ ദ\u200dര്\u200dശിച്ചാലും. <EOS>\n', 'പാണ്ഡവ സൈന്യത്തില്\u200d ഭീമാര്\u200dജുനതുല്യരും ശൂരരും വലിയ വില്ലാളികളുമായ യുയുധാനനും വിരാടനും മഹാരഥനായ ദ്രുപദനും ദൃഷ്ടകേതുവും ചേകിതാനനും വീര്യവാനായ കാശിരാജാവും പുരുജിത്തും കുന്തിഭോജനും നരശ്രേഷ്ടനായ ശൈബ്യനും വിക്രമിയായ യുധാമന്യുവും വീര്യവാനായ ഉത്തമൌജസും സുഭദ്രാതനയനായ അഭിമന്യുവും ദ്രൌപദീപുത്രന്മാരും ഉണ്ട്. അവര്\u200d എല്ലാവരും തന്നെ മഹാരഥന്മാരാണല്ലോ. <EOS>\n', 'ധൃഷ്ടകേതു, ചേകിതാനൻ, കാശിരാജാവ്, പുരുജിത്, കുന്തി ഭോജൻ, 

In [116]:
translator(encoder1, attn_decoder1)

ധൃതരാഷ്ട്രര്‍ ചോദിച്ചു: സഞ്ജയ, ധ‍ര്‍മ്മ ക്ഷേത്രമായ കുരുക്ഷേത്രത്തില്‍ ഒരുമിച്ചു ചേര്‍ന്നവരും യുദ്ധം ചെയ്യാന്‍ കൊതിക്കുന്നവരുമായ എന്റെ പുത്രന്‍മാരും പാണ്ഡവന്‍മാരും എന്ത് ചെയ്തു? <EOS>


In [40]:
%cd C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
import nltk
with open('test_malayalam.txt', 'r',  encoding="utf8") as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r',  encoding="utf8") as f: # translated Malayalam file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

bleu = nltk.bleu_score.corpus_bleu(reference, candidate )

print("bleu", bleu)

C:\Users\Glitch\Sanskrit2mal\parallel-corpus\sanskrit-mal
bleu 1.2478435706819264e-231


In [4]:
score_ref_a = bleu.corpus_bleu([reference], [candidate])
print("1 document with 1 reference sentence: {}".format(score_ref_a))

AttributeError: 'float' object has no attribute 'corpus_bleu'

In [41]:
from nltk.translate.bleu_score import sentence_bleu
with open('test_malayalam.txt', 'r',  encoding="utf8") as f: # ground truth file
    reference = [x.replace('\n', '') for x in f.readlines()]

with open('translated_malayalam.txt', 'r',  encoding="utf8") as f: # translated Malayalam file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
    
print('BLEU score -> {}'sentence_bleu(reference, candidate ))

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_29456/1160273461.py, line 8)